In [16]:
from llama_index.core import (SimpleDirectoryReader,
                         ServiceContext,
                         KnowledgeGraphIndex)
#
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.storage_context import StorageContext
from pyvis.network import Network
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
documents = SimpleDirectoryReader("data/paul_graham/source_files").load_data()
print(len(documents))

1


In [22]:
llm = OpenAI(temperature=0, model="gpt-4")
Settings.llm = llm
Settings.chunk_size = 512

In [23]:
from llama_index.core import StorageContext

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# https://docs.llamaindex.ai/en/stable/examples/index_structs/knowledge_graph/KnowledgeGraphDemo/
# default use LLM to generate graph triplets (https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/indices/knowledge_graph/base.py)
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
)

In [30]:
index.storage_context.persist(persist_dir='./data/paul_graham_KG')

In [24]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)
response = query_engine.query(
    "Tell me more about Interleaf",
)

In [27]:
response.response

"Interleaf is a company known for creating software for document creation. They also added a scripting language to their offerings. However, they faced significant challenges due to Moore's law."

In [28]:
response

Response(response="Interleaf is a company known for creating software for document creation. They also added a scripting language to their offerings. However, they faced significant challenges due to Moore's law.", source_nodes=[NodeWithScore(node=TextNode(id_='6b7227ce-51b4-4d1d-83fb-8eea0e91803a', embedding=None, metadata={'kg_rel_texts': ["['Interleaf', 'Made', 'Software for creating documents']", "['Interleaf', 'Added', 'Scripting language']", '[\'Interleaf\', \'Got crushed by\', "Moore\'s law"]'], 'kg_rel_map': {'more': [], 'Interleaf': [['Interleaf', 'Made', 'Software for creating documents'], ['Interleaf', 'Added', 'Scripting language'], ['Interleaf', 'Got crushed by', "Moore's law"]]}}, excluded_embed_metadata_keys=['kg_rel_map', 'kg_rel_texts'], excluded_llm_metadata_keys=['kg_rel_map', 'kg_rel_texts'], relationships={}, text='The following are knowledge sequence in max depth 2 in the form of directed graph like:\n`subject -[predicate]->, object, <-[predicate_next_hop]-, objec

In [29]:
from pyvis.network import Network

g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("kg/example.html")

example.html


In [31]:
documents = SimpleDirectoryReader("./documents").load_data()
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index2 = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
)

/Users/davideliu/miniconda3/envs/llm_testing/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [32]:
g = index2.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("kg/example2.html")

example2.html
